<a href="https://colab.research.google.com/github/xilo94/Big_Data_ETL/blob/main/part_one_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-23 18:08:27--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.50MB/s    in 0.2s    

2022-12-23 18:08:27 (5.50 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://bigetlxm.s3.amazonaws.com/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv"
spark.sparkContext.addFile(url)

# df = spark.read.format("tsv").option("header", "true").load("csvfile.csv")

userdf = spark.read.csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv"), sep="\t", header=True, inferSchema=True)




In [5]:
# Get the number of rows in the DataFrame.
# Show DataFrame
userdf.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

# Transform the Data

## Create the "review_id_table".

In [6]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

review_id_df = userdf.select("marketplace","customer_id", "review_id", "review_headline", "review_date")

review_id_df.show()

+-----------+-----------+--------------+--------------------+-----------+
|marketplace|customer_id|     review_id|     review_headline|review_date|
+-----------+-----------+--------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|Good quality. Shi...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|          Five Stars| 2015-08-31|
|         US|   19917519|R14Z1VR1N0Z9G6|It's really nice ...| 2015-08-31|
|         US|   18277171| R25ZRJL0GH0U0|           Two Stars| 2015-08-31|
|         US|    2593270|R3837KYH7AZNIY|          Four Stars| 2015-08-31|
|         US|    2592955|R2MN0QYCY6EVIV|      not bad at all| 2015-08-31|
|         US|   15168265|R3AN2UJ1D42ZS0|Mild, enjoyable t...| 2015-08-31|
|         US|   13761624|R3U29ZLUWEEK4M|          Five Stars| 2015-08-31|
|         US|   37070734|R16ZDMJJHK796C|        High Quality| 2015-08-31|
|         US|   29615023| RRRDOEJZD1Y22|          we like it| 2015-08-31|
|         US|   47893062|R2KR5ZEAS859D

In [7]:
review_id_df = review_id_df.dropna() 

## Create the "products" Table

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 

products_df = userdf.select("product_id", "product_title")
products_df = products_df.dropna()
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00OYRW4UE|Elite Sportz Exer...|
|B0000537JQ|     Ezy Dose Weekly|
|B00HXXO332|Pulse Oximeter, B...|
|B00EOB0JA2|SE Tools Tool Kit...|
|B00OC2O1UC|doTERRA HD Clear ...|
|B00HES9CMS|Viva Naturals #1 ...|
|B0016BFR4G|Uncle Lee's Organ...|
|B00K504UUG|Syrtenty Electrod...|
|B00HES9CMS|Viva Naturals #1 ...|
|B00P6TUO5G|Viva Naturals Org...|
|B0006VJ6TO|Body Back Company...|
|B00H9L7VIW|boostULTIMATE - 6...|
|B00P1JNZUW|doTERRA Veráge Sk...|
|B00N5HD340|      Straight Razor|
|B0007DHMVK|BONGER Massage To...|
|B0002JG2NI|Home Health Casto...|
|B00OYRW4UE|Elite Sportz Exer...|
|B00598WT0C|Aloha Eyewear “St...|
|B0006VJ6TO|Body Back Company...|
|B00J7G96NK|Oral B Glide Flos...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [9]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 

customers_df = userdf.select("customer_id", "review_id", "review_headline",)


In [10]:
# Grouping the data by the customer_id column by the number of times a customer reviewed a product

customers_df = customers_df.groupBy('customer_id').agg({"review_id": "count"})
customers_df.show()


+-----------+----------------+
|customer_id|count(review_id)|
+-----------+----------------+
|   11160821|               1|
|    1099354|               1|
|   37200466|               1|
|   31581262|               1|
|   27596904|               1|
|    2405598|               1|
|   11229098|               1|
|   17237226|               1|
|   46000147|               1|
|   15234640|               1|
|   28937218|               1|
|   43920023|               1|
|   27954895|               1|
|    8269764|               1|
|   31689770|               1|
|   30117841|               1|
|   38591899|               1|
|   24557051|               1|
|    2780908|               1|
|   32466409|               1|
+-----------+----------------+
only showing top 20 rows



## Create the "vine_table".

In [11]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = userdf.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1QX6706ZWJ1P5|          5|            0|          0|   N|
|R3QWMLJHIW6P37|          5|            0|          0|   N|
|R14Z1VR1N0Z9G6|          5|            1|          1|   N|
| R25ZRJL0GH0U0|          2|            0|          0|   N|
|R3837KYH7AZNIY|          4|            0|          1|   N|
|R2MN0QYCY6EVIV|          5|            0|          0|   N|
|R3AN2UJ1D42ZS0|          5|            0|          0|   N|
|R3U29ZLUWEEK4M|          5|            0|          0|   N|
|R16ZDMJJHK796C|          5|            0|          0|   N|
| RRRDOEJZD1Y22|          4|            0|          0|   N|
|R2KR5ZEAS859DK|          5|            0|          0|   N|
| RR7PGQY763IHF|          5|            0|          0|   N|
| RN37YYZBP9BMP|          5|            4|          4|   N|
| R1UXGB70OR5TW|          5|            

# Load

In [12]:
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cyfynzqdeakg.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", "password": "postgres", "driver":"org.postgresql.Driver"}

In [13]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id', mode=mode, properties=config)

In [14]:
# Write products_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='products_df', mode=mode, properties=config)

In [15]:
# Write customers_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='customers_df', mode=mode, properties=config)

In [16]:
# Write vine_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='vine_df', mode=mode, properties=config)